In [88]:
from concurrent.futures import ProcessPoolExecutor
# from loky import ProcessPoolExecutor  # for Windows users

def parallel(func, iterable):
    e = ProcessPoolExecutor()
    return e.map(func, iterable)

In [89]:
from pathlib import Path
import os

In [90]:
image_file_extensions = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def is_image_path_valid(path: Path):
    return path.is_file() and path.suffix in image_file_extensions

def verify_image(fn):
    "Confirm that `fn` can be opened"
    try:
        im = Image.open(fn)
        im.draft(im.mode, (32,32))
        im.load()
        return True
    except: return False

def load_image_file(path):
    return Image.open(path)

In [91]:
def load_images_recursively(root_dir: Path):
    ls = os.listdir
    
    images = []
    label2image = []
    
    def append_if_image(root: Path, filename: str):
        path = root / filename
        
        if is_image_path_valid(path):
            images.append(path)
            label2image.append(root.stem)
        
    for filename in ls(root_dir):
        file_path = root_dir / filename
            
        if file_path.is_dir():
            for nested_filename in ls(file_path):
                append_if_image(file_path, nested_filename)
        else:
            append_if_image(root_dir, filename)
            
    return images, label2image

In [92]:
from glob import glob
from pathlib import Path

In [93]:
sample_paths = [Path(g) for g in glob("./data/new_image_crops/*")]

In [94]:
len(sample_paths)

595

In [95]:
import numpy as np

In [96]:
from PIL import Image
import numpy as np

mean_rgb = (131.0912, 103.8827, 91.4953)

def load_image_for_feature_extraction(path='', shape=None):
    '''
    Referenced from VGGFace2 Paper:
    Q. Cao, L. Shen, W. Xie, O. M. Parkhi, and A. Zisserman, “VGGFace2: A dataset for recognising faces across pose and age,” arXiv:1710.08092 [cs], May 2018
    '''
    short_size = 224.0
    crop_size = shape
    img = Image.open(path)
    im_shape = np.array(img.size)    # in the format of (width, height, *)
    img = img.convert('RGB')

    ratio = float(short_size) / np.min(im_shape)
    img = img.resize(size=(int(np.ceil(im_shape[0] * ratio)),   # width
                           int(np.ceil(im_shape[1] * ratio))),  # height
                     resample=Image.BILINEAR)

    x = np.array(img)  # image has been transposed into (height, width)
    newshape = x.shape[:2]
    h_start = (newshape[0] - crop_size[0])//2
    w_start = (newshape[1] - crop_size[1])//2
    x = x[h_start:h_start+crop_size[0], w_start:w_start+crop_size[1]]
    
    # normalize colors to prevent overfitting on color differences 
    x = x - mean_rgb
    
    # returns transformed image, and original image
    return x

In [97]:
import warnings
image_size = (224,224,3)

np.random.seed(67)

def generate_batch(batch_size=16, shuffle=True):
    total_samples = len(sample_paths)
    
    if shuffle:
        idx = np.random.permutation(total_samples)
    else:
        idx = np.arange(total_samples)
        
    
    for ndx in range(0, total_samples, batch_size):
        batch_start = ndx
        batch_end = np.min([ndx + batch_size, total_samples])
        batch_idx = idx[batch_start: batch_end]
        
        batch_paths = np.array(sample_paths)[batch_idx]
        
        batch_images = []
        batch_image2idx = []
               
        for i, (nid, path) in enumerate(zip(batch_idx, batch_paths)):
            sub_image_paths = os.listdir(path)
            
            if(len(sub_image_paths) != 2):
                warnings.warn(f"{path} has {len(sub_image_paths)} files")
            else:
                
                batch_images.append(load_image_for_feature_extraction(path / sub_image_paths[0], image_size))
                batch_images.append(load_image_for_feature_extraction(path / sub_image_paths[1], image_size))
                batch_image2idx.append(nid)
                batch_image2idx.append(nid)
        
        yield np.stack(batch_images), np.stack(batch_image2idx), (batch_start, batch_end)

In [98]:
from saved_model.prepare_resnet50 import prepare_resnet_model

resnet_model = prepare_resnet_model("./saved_model/resnet50_ft_weight.pkl")

In [99]:
# Randomly split training and testing datasets
np.random.seed(67)

num_of_samples = len(sample_paths)
print(f"Total number of samples: {num_of_samples}")

idx = np.random.permutation(range(num_of_samples))
cut = int(0.8 * num_of_samples)
train_idx = idx[:cut]
valid_idx = idx[cut:]

Total number of samples: 595


In [100]:
from saved_model.prepare_resnet50 import prepare_resnet_model

resnet_model = prepare_resnet_model("./saved_model/resnet50_ft_weight.pkl")

In [101]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def extract_features(x):
    x = torch.Tensor(x.transpose(0, 3, 1, 2))  # nx3x224x224
    x = x.to(device)
    x = resnet_model(x).detach().cpu().numpy()
    return x

In [102]:
batch_images, batch_image2idx = next(iter(generate_batch()))

ValueError: too many values to unpack (expected 2)

In [ ]:
features = extract_features(batch_images)

In [103]:
features.shape

(1190, 2048)

In [104]:
batch_image2idx

array([186, 186, 241, 241, 478, 478])

In [105]:
from tqdm import tqdm
import math

In [106]:
num_of_features = 2048
features = np.zeros((num_of_samples * 2, num_of_features))
features2idx = np.empty(num_of_samples * 2)
labels = np.empty(num_of_samples * 2)
batch_size = 16

num_of_batches = math.ceil(num_of_samples / batch_size)

for batch_images, batch_image2idx, batch_num in tqdm(generate_batch(batch_size=16, shuffle=True), total=num_of_batches):
    batch_features = extract_features(batch_images)
    
    batch_start, batch_end = batch_num
    batch_start = batch_start * 2
    batch_end = np.min([batch_end * 2, num_of_samples * 2]) # because we are effectively doubling the batch_size 
    
    features[batch_start:batch_end, :] = batch_features
    features2idx[batch_start:batch_end] = batch_image2idx
    
    labels[batch_start:batch_end] = np.resize(np.arange(2), batch_end - batch_start)

100%|███████████████████████████████████████████| 38/38 [00:04<00:00,  7.94it/s]


In [107]:
features.shape

(1190, 2048)

In [108]:
features

array([[5.39536960e-02, 1.24970183e-01, 7.49640465e-02, ...,
        3.43718678e-01, 0.00000000e+00, 1.10962498e+00],
       [0.00000000e+00, 2.76269726e-02, 2.41232300e+00, ...,
        0.00000000e+00, 2.37279248e+00, 2.05646253e+00],
       [2.53713071e-01, 3.84531766e-02, 1.43994594e+00, ...,
        1.73197722e+00, 7.25936413e-01, 7.65277743e-02],
       ...,
       [5.05050468e+00, 9.18740177e+00, 2.82824683e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.59333497e-02],
       [1.85876331e+01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.94361019e+00, 0.00000000e+00],
       [5.48500729e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.59657001e+00, 3.46146274e+00, 0.00000000e+00]])

In [109]:
## Backup
np.save("./data/new_features.npy", features)

In [132]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(alpha=0.0001, penalty='elasticnet',loss='log')

In [133]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [134]:
model.fit(x_train, y_train)

SGDClassifier(loss='log', penalty='elasticnet')

In [135]:
y_pred = model.predict(x_test)

In [136]:
y_pred

array([0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
       0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0.,
       0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
       1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1.,
       0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
       0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 0., 0.

In [137]:
def binary_accuracy(pred, actual):
    num_correct = (pred == actual).sum()
    return num_correct / len(actual)

In [138]:
binary_accuracy(y_pred, y_test)

0.5210084033613446